# Playlist analyzer

Because of what happened with my account being flagged, this current code doesn't work. It was working perfectly until class, but I will have to find a workaround that doesn't use the api to find song data on the user's playlist. 

## Import the spotify api

In [76]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Define your credentials and redirection URI
SPOTIPY_CLIENT_ID = 'dda606479dd244a9b0948c01f155ac7a'
SPOTIPY_CLIENT_SECRET = 'd714739739a04b0fa662e521e1a85ff8'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'

# Create the Spotify client

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
))

user_name = input("Enter your spotify ID: ")
playlists = spotify.user_playlists(user_name)
name = input("Enter playlist name: ")


# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
def get_playlist_tracks(playlist_id):
    results = spotify.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks

playlist_id = 0
for i in playlists["items"] : 
    if i["name"] == name :
        playlist_id = i["id"]
playlist_data = get_playlist_tracks(playlist_id)


## Extract song data on each song

In [77]:
playlist = []
names = []

for i in playlist_data : 
    names.append(i["track"]["name"])
    playlist.append(spotify.audio_features(i["track"]["id"])[0])


### Convert the new data into a dataframe including the name of each song

In [78]:
collumns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"]
selected_columns = ["energy", "acousticness", "valence", "speechiness"]

df = pd.DataFrame(playlist, columns=collumns)
df = df[selected_columns]
df["name"] = names

## Find the distance of each song to the central point of the playlist's cluster

In [79]:
model = KMeans(n_clusters=1)
model.fit(df[selected_columns])


centroid = model.cluster_centers_[0]
distances = []

for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i]], [centroid], metric="euclidean")
    distances.append(distance[0][0])

df["distance"] = distances

Q1 = df["distance"].quantile(0.25)
Q3 = df['distance'].quantile(0.75)

max = Q3 + (Q3-Q1)*2
print(max)

df.loc[df["distance"] > max]



0.6861360285927631


,energy,acousticness,valence,speechiness,name,distance
56,0.236,0.898,0.254,0.0318,Moon,0.869987
198,0.846,0.720,0.747,0.0741,PROBLEMATIC,0.695793
218,0.398,0.872,0.482,0.2680,Hex,0.763450


## Load in the data

In [80]:
df = pd.read_csv("data.csv")
df['energy'] = pd.to_numeric(df['energy'], errors='coerce')


/var/folders/k6/qgdgj9qj743ff3dn87fyv0yr0000gn/T/ipykernel_2575/3618754505.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


## Find recommendations

In [81]:
recommendations = []
for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i].values], [centroid], metric="euclidean")
    if (distance < 0.2) :
        title = df.iloc[i]["title"]
        if ("(" not in title) : 
            recommendations.append(title)
    if len(recommendations) >= 50 :
        break

print(recommendations)



['Psycho', 'Slowdown', 'Nothing Since Then', 'Drunken Pirates', 'Bitch', 'Light in Me', 'Light in me', 'Kiss You', 'Blue', 'Good', 'Combative Five', 'Cooked', 'Hard to find', 'one', 'Wood West', 'Euphoria', 'Sting fells', 'Wather', 'Fly', 'Hard Love', 'Ban Tay Lanh', 'Buoc Di', 'Buon Lam Gi', 'Canh En', 'Gia Quen Em', 'Hoa Roi Cua Phat', 'Rainy Day Memories', 'Tang Phuc', 'Troi Cung Em', 'Troi Nguoc', 'Trom Tim Ai', 'Xa Em', 'Bust the New Jam', 'Bust The New Jam', 'Stay', 'Titanium', 'Play Hard', 'Stay The Night', 'Secrets', 'Lost In Cheeseville', "Don't Stop the Music", 'Tainted Love', "You're Still the One", 'Move Your Body', 'You & I feat. Tina Amy', 'You & I', 'Not Quite Human', 'Shapeshifter', 'Photographic', 'Sacrilege']
